In [4]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation
from pgmpy.factors.discrete import DiscreteFactor

from utils import *
import pandas as pd
import numpy as np

# Algoritmo genetico para obtener la mejor combinacion de medicamentos dada una patologia



** Setear los parametros del individuo (background, sintomas, efectos secundarios) **

In [5]:
class PathologyIndividual:
    "Clase destinada a las patologias, antecedentes del paciente, sintomas de la patologia y efectos de los medicamentos "

    def __init__(self, pathology, diccBackground, diccMedicines,arrStates,arrSymtopm, arrWeigth, arrEffects, bayesianNetwork):
        self.pathology = pathology
        self.diccBackground ={}
        self.diccMedicines  ={}
        
        self.arrStates       = arrStates[:]
        self.arrSymtopm      = arrSymtopm[:]
        self.arrWeigth       = arrWeigth[:]
        self.arrEffects      = arrEffects[:]
        self.bayesianNetwork = bayesianNetwork
        
        for index in diccBackground:  
            self.diccBackground[index]  = diccBackground[index]
        
        for index in diccMedicines:           
            self.diccMedicines[index]  = diccMedicines[index]



**Clase abstracta de un individuo de algoritmo genético**

In [6]:
class Individual:
    "Clase abstracta para individuos de un algoritmo evolutivo."

    def __init__(self, chromosome):
        self.chromosome = chromosome

    def crossover(self, other):
        "Retorna un nuevo individuo cruzando self y other."
        raise NotImplementedError
        
    def mutate(self):
        "Cambia los valores de algunos genes."
        raise NotImplementedError

<b>Clase concreta de un individuo del problema de combinacion de medicamentos</b>

In [30]:
import random

class Individual_medicines(Individual):
    "Clase que implementa el individuo en el problema de la busqueda de medicamentos que maximice la ausencia de los sintomas."

    def __init__(self, chromosome):
        self.chromosome = chromosome[:]
        self.fitness    = -1

    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre self y other "
        #print("Entro a onepoint")
        c = random.randrange(len(self.chromosome))
        #print(self.chromosome)
        #print(other.chromosome)
        #print(c)
        ind1 = Individual_medicines(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual_medicines(other.chromosome[:c] + self.chromosome[c:])
        #print("Primer hijo")
        #print(ind1.chromosome)
        #print("Segundo hijo")
        #print(ind2.chromosome)
        return [ind1, ind2]   
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual_medicines(chromosome1)
        ind2 = Individual_medicines(chromosome2)
        return [ind1, ind2]     

    def mutate_position(self, arrStates):
        "Cambia aleatoriamente el estado del medicamento (prescribir, prescribir bajas condiciones, no prescribir)"
        #print("Entro a mutate")
        for i,val in enumerate(self.chromosome):
            if (random.uniform(0, 1) < 0.25):
                new_value = changeState(val, arrStates[i])
                self.chromosome[i] = new_value
                #print(self.chromosome[i])       
        #print(self.chromosome)
        return self.chromosome
    

In [24]:
def changeState(val, numStates):
    arrStates = []
    for i in range(0,numStates):
        arrStates.append(i)
    arrStates.remove(val)
    new_value = random.choice(arrStates)
    return new_value
    

<b>Funcion de fitness para evaluar un individuo del problema de combinacion de medicamentos</b>

In [9]:
def createEvidence(dicc1, dicc2):
    evidence = dicc1.copy()
    evidence.update(dicc2)
    return evidence

In [10]:
def setStatesChromosome(diccMedicines, individuo):
    chromosome = individuo.chromosome
    n = len(chromosome)
    i = 0
    for key in diccMedicines:
        diccMedicines[key] = chromosome[i]
        i+=1
    return diccMedicines
        

In [11]:
def fitnessFunction(params):
    """Retorna el fitness de un cromosoma en el problema de combinacion de medicamentos """
    index, pathologyInd, individuo = params
    
    bp = BeliefPropagation(pathologyInd.bayesianNetwork)
    mergeSymEff      = pathologyInd.arrSymtopm + pathologyInd.arrEffects  
    
    newDiccMedicines = setStatesChromosome(pathologyInd.diccMedicines, individuo) 
    
    evidence = createEvidence(pathologyInd.diccBackground,pathologyInd.diccMedicines) 
    
    phi      = bp.query(variables=mergeSymEff, evidence = evidence)
    sumAbsen = 0
    sumTotal = 0
    
    for i,val in enumerate(mergeSymEff):
        arrProb = phi[val].values
        large = len(arrProb)
        sumAbsen += arrProb[large-1]*pathologyInd.arrWeigth[i]
        sumTotal += 1*pathologyInd.arrWeigth[i]  
    fitness = sumAbsen / sumTotal
    #print(fitness)  
    individuo.fitness = fitness
    print(individuo.fitness)  
    #fitness = sumAbsen / sumTotal
    return [index, fitness]
    

<b>Funcion para evaluar toda una población de individuos con la funcion de fitnes especificada</b>

In [12]:
import multiprocessing

In [13]:
def evaluate_population(pathologyInd, population, fitness_fn):
    """ Evalua una poblacion de individuos con la funcion de fitness pasada """
    popsize    = len(population)

    pool_size = 10
    param_arr = []
    for i in range(popsize):
        if population[i].fitness == -1:    # si el individuo no esta evaluado
            param_arr.append([i, pathologyInd, population[i]])

    pool = multiprocessing.Pool(pool_size)
    pool_results = pool.map(fitness_fn, param_arr)
    
    # assign results to population
    for fitness_arr in pool_results:
        population[fitness_arr[0]].fitness = fitness_arr[1]
    pool.close()
    pool.join()

    #for i in range(len(population)):
    #    print(population[i].fitness)

<b>Funcion que selecciona con el metodo de torneo un par de individuos de population para cruzamiento </b>

In [14]:
def getArrayRivals(popsize,ts):
    rival = []
    for i in range(ts): 
        rival_index = random.randint(0,popsize-1)
        if rival_index not in rival: 
            rival.append(rival_index)
    return rival

In [15]:
def chooseParent(population,ts):
    popsize = len(population)    
    rivalParent = getArrayRivals(popsize,ts)   
    #print(rivalParent)
    maxFitness = 0
    iParent=0
    for index in rivalParent:
        #print(population[index].fitness)
        if (population[index].fitness > maxFitness):
            maxFitness = population[index].fitness 
            iParent = index
    #print(iParent)        
    return iParent
    

In [16]:
#Seleccion de los mejores individios o conjuntos de medicamentos
def select_parents_tourney(population,ts):
    population_select = population.copy()
    # Escoje el primer padre
    iParent1 = chooseParent(population_select,ts)
    #Desconsiderar el padre ya escogido
    population_select.pop(iParent1)
    # Escoje el segundo padre
    iParent2 = chooseParent(population_select,ts)    
      
    return (population[iParent1], population[iParent2])

<b>Funcion que selecciona sobrevivientes para la sgte generacion, dada la poblacion actual y poblacion de hijos </b>

In [17]:
def select_survivors(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:numsurvivors]
    for i in range(numsurvivors): next_population.append(population[isurvivors[i]])
    return next_population

<b>Algoritmo Genetico</b>   
Recibe una clase parametro para el individuo, funcion de fitness, numero de generaciones (ngen) y taza de mutación (pmut)

In [27]:
def genetic_algorithm(pathologyInd, population, fitness_fn, ngen=100, pmut=0.1,ts=2):
    "Algoritmo Genetico "
    
    popsize    = len(population)
    evaluate_population(pathologyInd,population, fitness_fn)  # evalua la poblacion inicial
    ibest       = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
    bestfitness = [population[ibest[0]].fitness]
    print("Poblacion inicial, best_fitness = {}".format(population[ibest[0]].fitness))
    print(population)
    for g in range(ngen):   # Por cada generacion
        print("Generacion" + str(g))
        ## Selecciona las parejas de padres para cruzamiento 
        mating_pool = []
        for i in range(int(popsize/2)): mating_pool.append(select_parents_tourney(population,ts)) 
        print('Luego del mating pool')
        print(mating_pool)
        ## Crea la poblacion descendencia cruzando las parejas del mating pool con Recombinación de 1 punto
        offspring_population = []
        for i in range(len(mating_pool)): 
            offspring_population+= mating_pool[i][0].crossover_onepoint(mating_pool[i][1])
            #offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) )
        print('Antes de la mutacion')
        print(offspring_population)
        
        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if random.uniform(0, 1) < pmut: 
                print(offspring_population[i])
                offspring_population[i] = offspring_population[i].mutate_position(pathologyInd.arrStates)
        
        print('Luego de la mutacion')
        print(offspring_population)
        ## Evalua la poblacion descendencia
        evaluate_population(pathologyInd, offspring_population, fitness_fn)  # evalua la poblacion inicial
        
        ## Selecciona popsize individuos para la sgte. generación de la union de la pob. actual y  pob. descendencia
        population = select_survivors(population, offspring_population, popsize)

        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        bestfitness.append(population[ibest[0]].fitness)
        print("generacion {}, best_fitness = {}".format(g, population[ibest[0]].fitness))
    
    return population[ibest[0]], bestfitness  

 <b>Algoritmo de Busqueda Genetica para el problema de combinacion de medicamentos en adultos mayores</b>   

In [28]:
def genetic_search_medicines(ts,pathologyName, background, medicines, states, symtopms, weights, effects, bayesianNetwork, fitness_fn, num_medicines=11, popsize=20, ngen=100, pmut=0.5):
    import random
    population = []

    ## Crea la poblacion inicial con cromosomas aleatorios
    for i in range(popsize):
        chromosome = [random.randint(0,1) for j in range(1,(num_medicines+1))]##aqui le quite el +1
        random.shuffle(chromosome)
        population.append( Individual_medicines(chromosome) )
    
    pathologyInd = PathologyIndividual(pathologyName, background, medicines,states, symtopms, weights, effects, bayesianNetwork)

    ## llama al algoritmo genetico para encontrar una solucion al problema de la combinacion de medicamentos
    return genetic_algorithm(pathologyInd, population, fitness_fn, ngen, pmut,ts)
        

# Probando el Algoritmo genetico

In [31]:
import matplotlib.pyplot as plt
from pgmpy.readwrite import BIFReader

reader = BIFReader('hipertension.bif')
bn0 = reader.get_model()
# busca solucion para el problema de medicamentos. Usa 10 individuos, 100 generaciones y taza de mutación de 0.25
best_ind, bestfitness = genetic_search_medicines(4, 'Hipertension Arterial',{'EP_Insuficiencia_Cardiaca':0}, {'MR_metropolol':1, 'MR_hidro':1,'MR_amlodipino':0,'MR_enalapril':0,'MR_captopril':0, 'MR_carvedilol':0,'MR_furosemida': 0,'MR_losartan':0, 'MR_prazosin':0,'MR_propanolol':0,'MR_verapamilo':0},[3,3,2,2,2,2,2,2,2,2,2],['SA_presion_arterial','SA_angina_de_pecho'], [0.5,1,1], ['SA_danho_rinhones'],bn0,fitnessFunction)
plt.plot(bestfitness)
plt.show()

0.446
0.446
0.286
0.446
0.406
0.446
0.406
0.446
0.446
0.446
0.486
0.446
0.446
0.446
0.446
0.446
0.486
0.446
0.446
0.406
Poblacion inicial, best_fitness = 0.48600000000000004
[<__main__.Individual_medicines object at 0x7f892f7cc828>, <__main__.Individual_medicines object at 0x7f892de09128>, <__main__.Individual_medicines object at 0x7f892f7c9748>, <__main__.Individual_medicines object at 0x7f892f7c9710>, <__main__.Individual_medicines object at 0x7f892f7cd588>, <__main__.Individual_medicines object at 0x7f892f7cdc88>, <__main__.Individual_medicines object at 0x7f892f7cdcf8>, <__main__.Individual_medicines object at 0x7f892f7cdac8>, <__main__.Individual_medicines object at 0x7f892f7cdcc0>, <__main__.Individual_medicines object at 0x7f892f7cdf98>, <__main__.Individual_medicines object at 0x7f892f7cdef0>, <__main__.Individual_medicines object at 0x7f892f7cdd30>, <__main__.Individual_medicines object at 0x7f892f7cdc50>, <__main__.Individual_medicines object at 0x7f892f7cd470>, <__main__.Ind

AttributeError: 'list' object has no attribute 'fitness'

In [15]:
pathologyInd = PathologyIndividual('Hipertension Arterial',{'EP_Insuficiencia_Cardiaca':0}, {'MR_metropolol':1, 'MR_hidro':1,'MR_amlodipino':0,'MR_enalapril':0,'MR_captopril':0, 'MR_carvedilol':0,'MR_furosemida': 0,'MR_losartan':0, 'MR_prazosin':0,'MR_propanolol':0,'MR_verapamilo':0},[3,3,2,2,2,2,2,2,2,2,2],['SA_presion_arterial','SA_angina_de_pecho'], [0.5,1,1], ['SA_danho_rinhones'],bn0)
    

In [16]:
pathologyInd.pathology

'Hipertension Arterial'

In [11]:
import random
population = []
num_medicines=11
for i in range(20):
    chromosome = [random.randint(0,1) for j in range(1,(num_medicines+1))]
    random.shuffle(chromosome)
    population.append( Individual_medicines(chromosome))
    print(Individual_medicines(chromosome).chromosome)

[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1]
[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]
[1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0]
[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1]
[1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]
[1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1]
[1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1]
[1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0]
[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0]
[1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
